# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Database Management

**Subtopic** - Basic CRUD Operations with a Python SQLite Database

**Use Case** - Create SQLite triggers to automatically log changes to a table in a Python application using sqlite3.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
##Problem Description:
- You are developing a secure data-tracking system for an enterprise application.
- Your task is to create an audit mechanism using SQLite triggers that automatically logs changes (INSERT, UPDATE, DELETE) made to a given table named employees.
- You must implement a Python function using the sqlite3 module that:
 - Initializes an in-memory SQLite database.
 - Creates two empty tables:
  - employees(id INTEGER PRIMARY KEY, name TEXT, role TEXT)
  - audit_log(id INTEGER PRIMARY KEY AUTOINCREMENT, action TEXT, employee_id INTEGER, name TEXT, role TEXT, timestamp TEXT)
 - Creates SQLite triggers to automatically:
  - Log every INSERT into employees as action 'INSERT'
  - Log every UPDATE into employees as action 'UPDATE'
  - Log every DELETE from employees as action 'DELETE'
- The timestamp should use datetime('now') from SQLite.
- Your function should return the contents of audit_log after performing the audit operations.

##Input Format:
- The function `setup_and_track_changes` accepts a single parameter:
 - operations: List[Tuple[str, Tuple]]
  - A list of operations to perform on the `employees` table.
  - Each operation is a tuple of the form (operation_type: str, data: tuple)
 - The allowed operation types are:
  - "INSERT" - Insert a new employee.
  - Format: ("INSERT", (id: int, name: str, role: str))
  - Example: ("INSERT", (1, "Alice", "Engineer"))
  
  - "UPDATE" — Update an existing employee's name and/or role.
  - Format: ("UPDATE", (id: int, new_name: Optional[str], new_role: Optional[str]))
  - Example: ("UPDATE", (1, "Alicia", "Senior Engineer"))

  - "DELETE" - Delete an existing employee.
  - Format: ("DELETE", (id: int,))
  - Example: ("DELETE", (2,))

##Output Format:
- Return a list of tuples, each representing a row from the audit_log table.
- Each tuple should have the following structure:
 - (log_id: int, action: str, employee_id: int, name: str, role: str, timestamp: str)
 - Where:
  - log_id: Auto-incremented primary key.
  - action: One of "INSERT", "UPDATE", or "DELETE".
  - employee_id: The ID of the employee affected.
  - name: Name of the employee at the time of the operation.
  - role: Role of the employee at the time of the operation.
  - timestamp: Timestamp of the operation using datetime('now') in ISO 8601 format.

##Function Signature:
```python
def setup_and_track_changes(operations: list[tuple[str, tuple]]) -> list[tuple]:
```

##Example:
###Input:
operations = [
    ("INSERT", (1, "Alice", "Engineer")),
    ("INSERT", (2, "Bob", "Manager")),
    ("UPDATE", (1, None, "Senior Engineer")),
    ("UPDATE", (2, "Robert", None)),
    ("DELETE", (1,))
]

###Output:
[
    (1, 'INSERT', 1, 'Alice', 'Engineer', '2025-07-08 15:00:00'),
    (2, 'INSERT', 2, 'Bob', 'Manager', '2025-07-08 15:00:00'),
    (3, 'UPDATE', 1, 'Alice', 'Senior Engineer', '2025-07-08 15:00:00'),
    (4, 'UPDATE', 2, 'Robert', 'Manager', '2025-07-08 15:00:00'),
    (5, 'DELETE', 1, 'Alice', 'Senior Engineer', '2025-07-08 15:00:00')
]


# Requirements
Explicit and Implicit Requirements:
- Use the sqlite3 module from Python’s standard library.
- Use an in-memory SQLite database (sqlite3.connect(":memory:")).
- Create exactly two tables: employees and audit_log with the specified schema.
- Create three SQLite triggers: one each for INSERT, UPDATE, and DELETE on the employees table.
- Use datetime('now') in triggers to populate the timestamp field in ISO 8601 format.
- Accept input as a list of operations in the form: ("INSERT", (id, name, role)), ("UPDATE", (id, new_name, new_role)), or ("DELETE", (id,)).
- Log INSERT and UPDATE operations using NEW.name and NEW.role, and DELETE operations using OLD.name and OLD.role.
- Return the audit_log table contents as a list of tuples ordered by the auto-incremented id column.
- Perform all employee operations after table and trigger creation.
- Prevent duplicate consecutive log entries for the same employee and action.
- For UPDATE operations, support partial updates where new_name or new_role may be None.
- Skip logging for UPDATE or DELETE operations when the employee ID does not exist.
- Do not allow direct INSERTs into audit_log; all entries must be generated via triggers.
- Ensure all logged data reflects the state of the employee row at the time of the operation.
- Ensure trigger-based logging works correctly even under rapid or successive modifications.
- Keep the function self-contained, reproducible (except for timestamps), and free of external dependencies.
- Maintain chronological accuracy in the audit log through correct trigger and execution ordering.
- If both new_name and new_role are None, no update or log is performed.

Solution Expectations:
- The `setup_and_track_changes` function must initialize an in-memory SQLite database using `sqlite3.connect(":memory:")`.
- It must create two tables: `employees` (id, name, role) and `audit_log` (id, action, employee_id, name, role, timestamp).
- It must define and register exactly three SQLite triggers: one each for INSERT, UPDATE, and DELETE on the `employees` table.
- Each trigger must insert a new row into `audit_log` using the appropriate values from `NEW` or `OLD`, along with `datetime('now')` for timestamp.
- All input operations must be executed in order, and corresponding changes must be logged automatically via triggers.
- The function must handle optional updates correctly (e.g., updating only name or only role).
- The final output must be a list of tuples representing the `audit_log` rows in ascending order of the `id` column.
- The implementation must not manually insert rows into `audit_log`; logging must rely entirely on the triggers.
- The function must use only the Python standard library, with `sqlite3` as the only database interface.
- The output must include at least one INSERT, one UPDATE, and one DELETE action reflected in the audit log.

Edge Cases and Behavior:
- Inserting multiple employees:
  - Each INSERT should create a separate entry in the audit_log with action 'INSERT'.

- Updating a non-existent employee:
  - No changes occur in the employees table or audit_log since the UPDATE does not affect any row.

- Deleting a non-existent employee:
  - No changes occur in the employees table or audit_log since the DELETE does not affect any row.

- Updating only the name or only the role:
  - The UPDATE should modify only the specified field while retaining the other, and log the complete updated row.

- Deleting all employees:
  - Each DELETE should generate a corresponding entry in audit_log with correct old values for each employee.

- Performing operations in rapid succession:
  - All entries should be logged in the correct order with timestamps close together (as per SQLite’s datetime('now')).

- Attempting to insert an employee with a duplicate id:
  - Should raise an SQLite integrity error due to PRIMARY KEY constraint; this should be considered an invalid operation.

- Manual audit_log insertion:
  - Must not occur in the solution; all entries must originate only from the triggers.

- Timestamps:
  - All timestamps must be in UTC format as per SQLite’s datetime('now'), and reflect the moment of operation.

In [ ]:
# code

"""Audit-tracked employee database using SQLite in-memory database."""

import sqlite3
from typing import List, Tuple


def setup_and_track_changes(
        operations: List[Tuple[str, Tuple]]) -> List[Tuple]:
    """
    Set up SQLite in-memory DB with triggers and execute operations.

    Args:
        operations: A list of tuples representing
                    database operations to perform.
                    Each tuple contains an operation type
                    and its corresponding data.

    Returns:
        A list of tuples representing rows in the audit_log table.
    """
    connection = sqlite3.connect(":memory:")
    cursor = connection.cursor()

    cursor.execute('''
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT,
            role TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE audit_log (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            action TEXT,
            employee_id INTEGER,
            name TEXT,
            role TEXT,
            timestamp TEXT
        )
    ''')

    cursor.execute('''
        CREATE TRIGGER insert_employee_trigger
        AFTER INSERT ON employees
        FOR EACH ROW
        BEGIN
            INSERT INTO audit_log(action, employee_id, name, role, timestamp)
            VALUES (
                'INSERT',
                NEW.id,
                NEW.name,
                NEW.role,
                datetime('now')
            );
        END;
    ''')

    cursor.execute('''
        CREATE TRIGGER update_employee_trigger
        AFTER UPDATE ON employees
        FOR EACH ROW
        WHEN NEW.name IS NOT OLD.name OR NEW.role IS NOT OLD.role
        BEGIN
            INSERT INTO audit_log(action, employee_id, name, role, timestamp)
            VALUES (
                'UPDATE',
                NEW.id,
                NEW.name,
                NEW.role,
                datetime('now')
            );
        END;
    ''')

    cursor.execute('''
        CREATE TRIGGER delete_employee_trigger
        AFTER DELETE ON employees
        FOR EACH ROW
        BEGIN
            INSERT INTO audit_log(action, employee_id, name, role, timestamp)
            VALUES (
                'DELETE',
                OLD.id,
                OLD.name,
                OLD.role,
                datetime('now')
            );
        END;
    ''')

    for operation in operations:
        op_type, data = operation
        if op_type == "INSERT":
            try:
                cursor.execute(
                    'INSERT INTO employees (id, name, role)'
                    ' VALUES (?, ?, ?)', data
                )
            except sqlite3.IntegrityError:
                raise
        elif op_type == "UPDATE":
            emp_id, new_name, new_role = data
            cursor.execute(
                'SELECT name, role FROM employees WHERE id = ?', (emp_id,)
            )
            result = cursor.fetchone()

            if result:
                current_name, current_role = result
                updated_name = new_name if (new_name
                                            is not None) else current_name
                updated_role = new_role if (new_role
                                            is not None) else current_role

                if new_name is None and new_role is None:
                    continue

                if (
                    updated_name != current_name
                    or updated_role != current_role
                ):
                    cursor.execute(
                        'UPDATE employees SET name = ?, role = ? WHERE id = ?',
                        (updated_name, updated_role, emp_id)
                    )
            else:
                continue
        elif op_type == "DELETE":
            emp_id_to_delete = data[0]
            cursor.execute(
                'SELECT id FROM employees WHERE id = ?', (emp_id_to_delete,)
            )
            if cursor.fetchone():
                cursor.execute(
                    'DELETE FROM employees WHERE id = ?', (emp_id_to_delete,)
                )
            else:
                continue

    connection.commit()
    cursor.execute(
        'SELECT id, action, employee_id, name, role, timestamp '
        'FROM audit_log ORDER BY id'
    )
    audit_log = cursor.fetchall()
    connection.close()

    return audit_log

In [ ]:
# tests

import unittest
import sqlite3
import re
import datetime

from main import setup_and_track_changes


class TestSetupAndTrackChanges(unittest.TestCase):
    """Comprehensive unit tests for the setup_and_track_changes function."""

    _TS_REGEX = re.compile(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}")

    def _check_timestamps(self, logs):
        """Assert each timestamp is ISO‑8601 and logs are chronological."""
        parsed = []
        for row in logs:
            self.assertRegex(row[5], self._TS_REGEX, "Timestamp format invalid")
            parsed.append(datetime.datetime.fromisoformat(row[5]))
        # Non‑decreasing order preserves audit chronology
        self.assertEqual(parsed, sorted(parsed), "Timestamps out of order")

    def test_insert_update_delete_logging(self):
        """Happy‑path: INSERT ➔ UPDATE ➔ DELETE for a single row."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("UPDATE", (1, "Alicia", "Senior Engineer")),
            ("DELETE", (1,)),
        ]
        logs = setup_and_track_changes(ops)

        self.assertEqual(len(logs), 3)
        self.assertEqual([r[1] for r in logs], ["INSERT", "UPDATE", "DELETE"])
        self.assertEqual([r[2] for r in logs], [1, 1, 1])

        self.assertEqual(logs[1][3:5], ("Alicia", "Senior Engineer"))
        self.assertEqual(logs[2][3:5], ("Alicia", "Senior Engineer"))

        self._check_timestamps(logs)

    def test_partial_update_role_only(self):
        """UPDATE that changes only the role keeps the existing name."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("UPDATE", (1, None, "Lead Engineer")),
        ]
        logs = setup_and_track_changes(ops)

        self.assertEqual(len(logs), 2)
        self.assertEqual(logs[1][1], "UPDATE")
        self.assertEqual(logs[1][3], "Alice")
        self.assertEqual(logs[1][4], "Lead Engineer")
        self._check_timestamps(logs)

    def test_partial_update_name_only(self):
        """UPDATE that changes only the name keeps the existing role."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("UPDATE", (1, "Alicia", None)),
        ]
        logs = setup_and_track_changes(ops)

        self.assertEqual(len(logs), 2)
        self.assertEqual(logs[1][3], "Alicia")
        self.assertEqual(logs[1][4], "Engineer")
        self._check_timestamps(logs)

    def test_update_nonexistent_employee(self):
        """Updating a missing row must leave the audit untouched."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("UPDATE", (99, "Ghost", "Phantom")),
        ]
        logs = setup_and_track_changes(ops)
        self.assertEqual(len(logs), 1)

    def test_delete_nonexistent_employee(self):
        """Deleting a missing row must leave the audit untouched."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("DELETE", (99,)),
        ]
        logs = setup_and_track_changes(ops)
        self.assertEqual(len(logs), 1)

    def test_duplicate_insert_error(self):
        """Inserting the same primary‑key twice should raise IntegrityError."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("INSERT", (1, "Bob", "Manager")),
        ]
        with self.assertRaises(sqlite3.IntegrityError):
            setup_and_track_changes(ops)

    def test_prevent_duplicate_consecutive_logs(self):
        """
        Two identical UPDATEs in a row must yield
        only one audit entry for the UPDATE action.
        """
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("UPDATE", (1, None, "Engineer II")),
            ("UPDATE", (1, None, "Engineer II")),
        ]
        logs = setup_and_track_changes(ops)
        self.assertEqual(len(logs), 2)
        self.assertEqual([r[1] for r in logs], ["INSERT", "UPDATE"])

    def test_timestamp_format_and_order(self):
        """All timestamps should be ISO‑8601 and in ascending order."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("UPDATE", (1, "Alicia", "Lead Engineer")),
            ("DELETE", (1,)),
        ]
        logs = setup_and_track_changes(ops)
        self._check_timestamps(logs)

    def test_skip_update_no_changes(self):
        """UPDATE with both new_name and new_role as None is a no‑op."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("UPDATE", (1, None, None)),
        ]
        logs = setup_and_track_changes(ops)
        self.assertEqual(len(logs), 1)

    def test_multiple_inserts_logging_order(self):
        """Multiple INSERTs must be logged in the order executed."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("INSERT", (2, "Bob", "Manager")),
            ("INSERT", (3, "Charlie", "Analyst")),
        ]
        logs = setup_and_track_changes(ops)
        self.assertEqual(len(logs), 3)
        self.assertEqual([r[2] for r in logs], [1, 2, 3])
        self.assertEqual([r[1] for r in logs], ["INSERT"] * 3)
        self._check_timestamps(logs)

    def test_update_trailing_whitespace_is_logged(self):
        """UPDATE with whitespace/case change should still be logged."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("UPDATE", (1, "Alice", "Engineer ")),
        ]
        logs = setup_and_track_changes(ops)
        self.assertEqual(len(logs), 2)
        self.assertEqual(logs[1][1], "UPDATE")

    def test_update_after_delete_is_ignored(self):
        """UPDATE after DELETE should not log anything."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("DELETE", (1,)),
            ("UPDATE", (1, "Ghost", "Phantom")),
        ]
        logs = setup_and_track_changes(ops)
        self.assertEqual(len(logs), 2)
        self.assertEqual([r[1] for r in logs], ["INSERT", "DELETE"])

    def test_redundant_delete_is_ignored(self):
        """Second DELETE on same ID should not log again."""
        ops = [
            ("INSERT", (1, "Alice", "Engineer")),
            ("DELETE", (1,)),
            ("DELETE", (1,)),
        ]
        logs = setup_and_track_changes(ops)
        self.assertEqual(len(logs), 2)
        self.assertEqual([r[1] for r in logs], ["INSERT", "DELETE"])


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/9e158151-07c2-4acb-8432-3ff39f4fbc8a?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67755%2Fview
#### Model code:

```python
# code generated by the model
import sqlite3
from typing import List, Tuple, Optional

def setup_and_track_changes(operations: List[Tuple[str, Tuple]]) -> List[Tuple]:
    connection = sqlite3.connect(":memory:")
    cursor = connection.cursor()

    # Create tables
    cursor.execute('''
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            role TEXT NOT NULL
        )
    ''')
    cursor.execute('''
        CREATE TABLE audit_log (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            action TEXT NOT NULL,
            employee_id INTEGER NOT NULL,
            name TEXT NOT NULL,
            role TEXT NOT NULL,
            timestamp TEXT NOT NULL
        )
    ''')

    # Create triggers
    cursor.execute('''
        CREATE TRIGGER insert_employee_trigger
        AFTER INSERT ON employees
        BEGIN
            INSERT INTO audit_log(action, employee_id, name, role, timestamp)
            VALUES ('INSERT', NEW.id, NEW.name, NEW.role, datetime('now'));
        END;
    ''')

    cursor.execute('''
        CREATE TRIGGER update_employee_trigger
        AFTER UPDATE ON employees
        BEGIN
            INSERT INTO audit_log(action, employee_id, name, role, timestamp)
            VALUES ('UPDATE', NEW.id, NEW.name, NEW.role, datetime('now'));
        END;
    ''')
    # <Issue>: This trigger does not include a WHEN clause to avoid logging unchanged updates.

    cursor.execute('''
        CREATE TRIGGER delete_employee_trigger
        AFTER DELETE ON employees
        BEGIN
            INSERT INTO audit_log(action, employee_id, name, role, timestamp)
            VALUES ('DELETE', OLD.id, OLD.name, OLD.role, datetime('now'));
        END;
    ''')

    # Execute provided operations
    for operation in operations:
        op_type, data = operation
        if op_type == "INSERT":
            cursor.execute('INSERT INTO employees (id, name, role) VALUES (?, ?, ?)', data)
        elif op_type == "UPDATE":
            emp_id, new_name, new_role = data
            cursor.execute('SELECT name, role FROM employees WHERE id = ?', (emp_id,))
            result = cursor.fetchone()
            if result:
                name, role = result
                updated_name = new_name if new_name is not None else name
                updated_role = new_role if new_role is not None else role
                if updated_name != name or updated_role != role:
                    cursor.execute('UPDATE employees SET name = ?, role = ? WHERE id = ?',
                                   (updated_name, updated_role, emp_id))
                    # <Issue>: If both new_name and new_role are None and match existing, this block does nothing (correct), but not explicitly skipping in that case
                # <Issue>: No check for case where employee does not exist (i.e., `result` is None) — should skip update silently per spec.
        elif op_type == "DELETE":
            cursor.execute('DELETE FROM employees WHERE id = ?', data)
            # <Issue>: No check whether employee exists before deleting. Should skip silently if employee doesn't exist (to prevent unnecessary DELETE that won’t trigger logging).

    # Retrieve audit log
    cursor.execute('SELECT id, action, employee_id, name, role, timestamp FROM audit_log ORDER BY id')
    audit_log = cursor.fetchall()

    connection.close()

    return audit_log
```